In [ ]:
%cd "/content/drive/MyDrive/Reverse Vending Machine ----------------------------------------------------------------"

/content/drive/MyDrive/Reverse Vending Machine ----------------------------------------------------------------


In [ ]:
!python3 ai.py --batch_size 32 --epoch 50 --lr 0.001 \
    --data_root "/content/drive/MyDrive/Reverse Vending Machine ----------------------------------------------------------------/dataset" \
    --save_dir "/content/drive/MyDrive/Reverse Vending Machine ----------------------------------------------------------------"

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/

In [ ]:
!python3 ai.py --batch_size 32 --epoch 10 --lr 0.001 \
    --data_root "/content/drive/MyDrive/Reverse Vending Machine/dataset" \
    --save_dir "/content/drive/MyDrive/Reverse Vending Machine" \
    --do_finetuning "True" 

In [ ]:
!python3 ai.py --batch_size 32 --epoch 10 --lr 0.01 \
    --data_root "/content/drive/MyDrive/Reverse Vending Machine/dataset" \
    --save_dir "/content/drive/MyDrive/Reverse Vending Machine" \
    --do_finetuning "False" \
    --continue_path "/content/drive/MyDrive/Reverse Vending Machine"

In [ ]:
import tensorflow as tf 
import tensorflow_hub as hub
import argparse
import numpy
import datetime

lr = 0.001
batch_size = 32
do_finetuning = True
epoch = 20

data_root = "/content/drive/MyDrive/Reverse Vending Machine ----------------------------------------------------------------/dataset"
classifier_model = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4"
datagen_kwargs = dict(rescale=1./255, brightness_range=(0.7, 1.0), vertical_flip=True, horizontal_flip=True, validation_split=0.2) # Pre-processing methods
img_shape = (224, 224)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_data = train_datagen.flow_from_directory(
    data_root,
    shuffle=True,
    target_size=img_shape,
    batch_size=batch_size,
    subset='training'
)
valid_data = train_datagen.flow_from_directory(
    data_root,
    shuffle=True,
    target_size=img_shape,
    batch_size=batch_size,
    subset='validation'
)

In [ ]:
# Raspberry pi ile son layer tuning lr = 0.01

In [ ]:
import tensorflow as tf 
import tensorflow_hub as hub
import argparse
import numpy
import datetime

lr = 0.001
batch_size = 32
do_finetuning = True
epoch = 20

data_root = "/content/drive/MyDrive/Reverse Vending Machine ----------------------------------------------------------------/dataset"
classifier_model = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4"
datagen_kwargs = dict(rescale=1./255, brightness_range=(0.7, 1.0), vertical_flip=True, horizontal_flip=True, validation_split=0.2) # Pre-processing methods
img_shape = (224, 224)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_data = train_datagen.flow_from_directory(
    data_root,
    shuffle=True,
    target_size=img_shape,
    batch_size=batch_size,
    subset='training'
)
valid_data = train_datagen.flow_from_directory(
    data_root,
    shuffle=True,
    target_size=img_shape,
    batch_size=batch_size,
    subset='validation'
)

In [7]:
model = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=img_shape+(3,)),
    # Pre-trained model from Tensorflow Hub 
    hub.KerasLayer(classifier_model, trainable=do_finetuning),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(train_data.num_classes, activation='softmax')
])

model.build((None,)+img_shape+(3,))

model.compile(optimizer=tf.keras.optimizers.Adam(lr=lr),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=['acc'])

steps_per_epoch = train_data.samples // train_data.batch_size
validation_steps = valid_data.samples // valid_data.batch_size

In [ ]:
model.summary()

In [ ]:
%cd "/content/drive/MyDrive/Reverse Vending Machine ----------------------------------------------------------------"
!mkdir checkpoints

In [ ]:
# Dont forget to clear out prior loggin data by "!rm -rf logs/image"
logdir = "logs/" + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
checkpoint_path = "/content/drive/MyDrive/Reverse Vending Machine ----------------------------------------------------------------/checkpoints"

# Define the basic TensorBoard callback.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

history = model.fit(train_data, epochs=epoch,
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_data,
    validation_steps=validation_steps,
    callbacks=[cp_callback, tensorboard_callback])

In [15]:
latest = tf.train.latest_checkpoint("/content/drive/MyDrive/Reverse Vending Machine/")
# Create a new model instance
model = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=img_shape+(3,)),
    # Pre-trained model from Tensorflow Hub 
    hub.KerasLayer(classifier_model, trainable=do_finetuning),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(train_data.num_classes, activation='softmax')
])

# Load the previously saved weights
model.load_weights(latest)

In [ ]:
%load_ext tensorboard

In [ ]:
# To see the logs after training, run this command
%tensorboard --logdir logs/

In [ ]:
import matplotlib.pylab as plt
def get_class_string_from_index(index):
   for class_string, class_index in valid_data.class_indices.items():
      if class_index == index:
         return class_string

x, y = next(valid_data)
image = x[0, :, :, :]
true_index = numpy.argmax(y[0])
plt.imshow(image)
plt.axis('off')
plt.show()

# Expand the validation image to (1, 224, 224, 3) before predicting the label
prediction_scores = model.predict(numpy.expand_dims(image, axis=0))
predicted_index = numpy.argmax(prediction_scores)
print("True label: " + get_class_string_from_index(true_index))
print("Predicted label: " + get_class_string_from_index(predicted_index))

In [16]:
# Converting model into TensorflowLite model
saved_model_dir = '/content/drive/MyDrive/Reverse Vending Machine ----------------------------------------------------------------/' # '[directory you want to save your model to]'
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# Saving the model and label map into two seperate files for further use
with open('model_finetune.tflite', 'wb') as f:
    f.write(tflite_model)

labels = '\n'.join(sorted(train_data.class_indices.keys()))

with open('labels.txt', 'w') as f:
    f.write(labels)

# All that is left is to use this TFLite model in your Raspberry Pi 4. 

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Reverse Vending Machine ----------------------------------------------------------------/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Reverse Vending Machine ----------------------------------------------------------------/assets


In [ ]:
%cd '/content/drive/MyDrive/Reverse Vending Machine ----------------------------------------------------------------/deneme'

/content/drive/MyDrive/Reverse Vending Machine ----------------------------------------------------------------/deneme


In [ ]:
import numpy as np
import matplotlib.pylab as plt

def set_input_tensor(interpreter, image):
  tensor_index = interpreter.get_input_details()[0]['index']
  input_tensor = interpreter.tensor(tensor_index)()[0]
  input_tensor[:, :] = image

def load_labels(path):
    """Loads the file path of the labels file."""
    with open(path, 'r') as f:
        return {i: line.strip() for i, line in enumerate(f.readlines())}

# Setting up the model and the labels
labels = load_labels("labels.txt")
interpreter = tf.lite.Interpreter("model_finetune.tflite")

error_count = 0
for _ in range(valid_data.samples):
    interpreter.allocate_tensors()
    _, height, width, _ = interpreter.get_input_details()[0]['shape']
    x, y = next(valid_data)
    image = x[0, :, :, :]
    true_index = np.argmax(y[0])

    top_k = 1
    set_input_tensor(interpreter, image)
    interpreter.invoke()
    output_details = interpreter.get_output_details()[0]
    output = np.squeeze(interpreter.get_tensor(output_details['index']))
    ordered = np.argpartition(-output, top_k)
    results = [(i, output[i]) for i in ordered[:top_k]]
    label_id, prob = results[0]

    if labels[true_index] != labels[label_id]:
        plt.imshow(image)
        plt.axis('off')
        plt.show()
        print("True label: " + labels[true_index] + " Probability: " + str(prob))
        print("Predicted label: " + labels[label_id])
        error_count += 1
print('Validation accuracy is {}'.format((1.0 - (error_count / valid_data.samples)) * 100.0))